In [1]:
import torch
from torch.utils.data import Dataset
import transformers
from transformers import BertForSequenceClassification, AdamW

# load the pretrained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# # Load the model from a specific epoch
checkpoint_path = "/home/deepali/codes/sentiment_analysis/Data/weights/model_epoch_1.pt"
# Load the saved state_dict
state_dict = torch.load(checkpoint_path, map_location=device)
# Remove unexpected keys (if any)
state_dict.pop("bert.embeddings.position_ids", None)
# Load the updated state_dict into the model
model.load_state_dict(state_dict, strict=False)
model.eval()
print("Model loaded successfully with adjusted state_dict.")

/home/deepali/anaconda3/envs/torch_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-25 16:09:28.410926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-25 16:09:28.500841: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-25 16:09:28.871363: W tensorflow/compiler/xla/stream_executor/platform/defa

Model loaded successfully with adjusted state_dict.


In [2]:
# export model to onnx
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

dummy_text = "This is a sample input for tensorrt conversion."
dummy_input = tokenizer(dummy_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
# move inputs to the device
dummy_input = {key: value.to(device) for key, value in dummy_input.items()}
# Define the ONNX export path
onnx_path = "/home/deepali/codes/sentiment_analysis/Data/model.onnx"

# export to onnx
# Export the model to ONNX
torch.onnx.export(
    model,  # The PyTorch model
    tuple(dummy_input.values()),  # Model inputs
    onnx_path,  # Output path
    input_names=list(dummy_input.keys()),  # Input names
    output_names=["output"],  # Output names
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "output": {0: "batch_size"},
    },  # Dynamic shapes
    opset_version=12,  # ONNX opset version
    do_constant_folding=True,  # Optimize constants
)

print(f"ONNX model exported to {onnx_path}")

ONNX model exported to /home/deepali/codes/sentiment_analysis/Data/model.onnx


In [3]:
import onnx

# Load and check the ONNX model
onnx_model = onnx.load("/home/deepali/codes/sentiment_analysis/Data/weights/model.onnx")
onnx.checker.check_model(onnx_model)
print("ONNX model is valid!")

ONNX model is valid!


In [4]:
!trtexec --onnx=/home/deepali/codes/sentiment_analysis/Data/weights/model.onnx\
        --saveEngine=/home/deepali/codes/sentiment_analysis/Data/weights/model.trt \
        --fp16  

/bin/bash: line 1: trtexec: command not found
